In [ ]:
# default_exp type_emb
# all_data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Type embeddings

## Variables

In [ ]:
VECS_DIR = '~/promo/socemb/data/vecs'

In [ ]:
LIMIT = 100_000

In [ ]:
model_1_spec = {'comm': 'askaconservative', 'year': 2019}

In [ ]:
model_2_spec = {'comm': 'askaconservative', 'year': 2020}

## Imports

In [ ]:
from socemb.read_data import *

In [ ]:
# export
import re
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

In [ ]:
from functools import reduce
import gensim

## Read comments

In [ ]:
f_path_1 = get_fpath_subr_yr(model_1_spec['comm'], LIMIT, model_1_spec['year'])

In [ ]:
f_path_2 = get_fpath_subr_yr(model_2_spec['comm'], LIMIT, model_2_spec['year'])

In [ ]:
comments_1 = read_comm_csv(f_path_1)

In [ ]:
comments_2 = read_comm_csv(f_path_2)

In [ ]:
comments_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97243 entries, 0 to 97242
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   body         97243 non-null  string        
 1   created_utc  97243 non-null  datetime64[ns]
 2   id           97243 non-null  string        
 3   subreddit    97243 non-null  string        
dtypes: datetime64[ns](1), string(3)
memory usage: 3.7 MB


## Split comments

### Split in temporal bins

In [ ]:
comments = comments.assign(date = pd.to_datetime(
    comments['created_utc'],
    errors='coerce'
))

In [ ]:
comments['date'].dt.year.value_counts()

In [ ]:
comments = comments[comments.date.dt.year == TIME]

### Split by communities

In [ ]:
comments\
    .value_counts('subreddit')\
    .head(20)

In [ ]:
comments = comments.query('subreddit == "politics"')

In [ ]:
comments = comments.query('subreddit == "AskHistorians"')

## Pre-process comments

In [ ]:
# export
def conv_to_lowerc(doc):
    return doc.lower()

In [ ]:
assert conv_to_lowerc('Test') == 'test'

In [ ]:
# export
def rm_punct(doc):
    return re.sub(r'[^\w\s]+', ' ', doc)

In [ ]:
assert rm_punct('No-punctuation!') == 'No punctuation '

In [ ]:
# export
def tokenize(doc):
    return doc.split()

In [ ]:
assert len(tokenize('These are three-tokens')) == 3

In [ ]:
# export
def detect_short_doc(doc, limit=10):
    if len(doc) >= limit:
        return False
    else:
        return True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei'], 10) == True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei', 'viere', 'fuenfe', 'sechse', 'simme', 'achte', 'neine', 'zehne'], 10) == False

In [ ]:
# export
def clean_docs(docs):
    docs_clean = docs\
        .apply(conv_to_lowerc)\
        .apply(rm_punct)\
        .apply(tokenize)\
        .where(lambda x : x.apply(detect_short_doc) == False)\
        .dropna()    
    return docs_clean

In [ ]:
docs_clean_1 = clean_docs(comments_1['body'])

In [ ]:
docs_clean_2 = clean_docs(comments_2['body'])

## Create corpus

In [ ]:
# export
class Corpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, docs_clean):
        self.docs_clean = docs_clean

    def __iter__(self):
        for doc in self.docs_clean:
            yield doc

In [ ]:
corpus_1 = Corpus(docs_clean_1)

In [ ]:
corpus_2 = Corpus(docs_clean_2)

## Train embeddings

In [ ]:
# export
def train_emb(corpus, MIN_COUNT=5, SIZE=300, WORKERS=8, WINDOW=3):
    model = Word2Vec(
        corpus, 
        min_count=MIN_COUNT,
        size=SIZE,
        workers=WORKERS, 
        window=WINDOW
    )
    return model

In [ ]:
model_1 = train_emb(corpus_1)

In [ ]:
%%time
model_2 = train_emb(corpus_2)

CPU times: user 45 s, sys: 223 ms, total: 45.2 s
Wall time: 12.3 s


## Evaluate embeddings

In [ ]:
for index, word in enumerate(model_1.wv.index2word):
    if index == 10:
        break
    print(f"{index}/{len(model_1.wv.index2word)}: {word}")

0/20738: the
1/20738: to
2/20738: and
3/20738: a
4/20738: of
5/20738: that
6/20738: is
7/20738: i
8/20738: it
9/20738: in


In [ ]:
model_1.wv.most_similar('person', topn=20)

[('man', 0.7230769991874695),
 ('woman', 0.7013160586357117),
 ('guy', 0.6996582746505737),
 ('someone', 0.6453497409820557),
 ('kid', 0.6225624084472656),
 ('baby', 0.6172075271606445),
 ('child', 0.6142028570175171),
 ('anyone', 0.6052091121673584),
 ('doctor', 0.5907690525054932),
 ('parent', 0.5853807926177979),
 ('fetus', 0.5693686604499817),
 ('politician', 0.5691657066345215),
 ('mother', 0.5550119876861572),
 ('somebody', 0.552615225315094),
 ('anybody', 0.5372178554534912),
 ('group', 0.5359237194061279),
 ('friend', 0.5345120429992676),
 ('citizen', 0.5333900451660156),
 ('lawyer', 0.5299038887023926),
 ('president', 0.528654932975769)]

In [ ]:
lex_rel_1 = 'person'
lex_rel_2 = 'man'
lex_unrel = 'time'

In [ ]:
sim_rel = model_1.wv.similarity(lex_rel_1, lex_rel_2)
sim_rel

0.723077

In [ ]:
sim_unrel_1 = model_1.wv.similarity(lex_rel_1, lex_unrel)
sim_unrel_1

0.19468208

In [ ]:
sim_unrel_2 = model_1.wv.similarity(lex_rel_2, lex_unrel)
sim_unrel_2

0.1299858

In [ ]:
assert sim_rel > sim_unrel_1

In [ ]:
assert sim_rel > sim_unrel_2

## Save model

In [ ]:
model_1.wv.save(f'{VECS_DIR}/model_1.kv')

In [ ]:
model_2.wv.save(f'{VECS_DIR}/model_2.kv')

## Load models

In [ ]:
model_1 = KeyedVectors.load(f'{VECS_DIR}/model_1.kv', mmap='r')

In [ ]:
model_2 = KeyedVectors.load(f'{VECS_DIR}/model_2.kv', mmap='r')

In [ ]:
# export
def load_model(SUBREDDIT: str, YEAR: int, LIMIT: int = 100_000):
    """Load word2vec model from disk."""
    model = KeyedVectors.load(f'data/vecs/{SUBREDDIT}_{YEAR}_{LIMIT}.kv', mmap='r')
    return model

In [ ]:
# export
def get_vec_from_model(lex: str, model):
    return model[lex]

## Align models

In [ ]:
def align_gensim_models(models, words=None):
    """
    Returns the aligned/intersected models from a list of gensim word2vec models.
    Generalized from original two-way intersection as seen above.
    
    Also updated to work with the most recent version of gensim
    Requires reduce from functools
    
    In order to run this, make sure you run 'model.init_sims()' for each model before you input them for alignment.
    
    ##############################################
    ORIGINAL DESCRIPTION
    ##############################################
    
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocabs = [set(m.vocab.keys()) for m in models]

    # Find the common vocabulary
    common_vocab = reduce((lambda vocab1,vocab2: vocab1&vocab2), vocabs)
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    
    # This was generalized from:
    # if not vocab_m1-common_vocab and not vocab_m2-common_vocab and not vocab_m3-common_vocab:
    #   return (m1,m2,m3)
    if all(not vocab-common_vocab for vocab in vocabs):
        print("All identical!")
        return models
        
    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: sum([m.vocab[w].count for m in models]),reverse=True)
    
    # Then for each model...
    for m in models:
        
        # Replace old vectors_norm array with new one (with common vocab)
        indices = [m.vocab[w].index for w in common_vocab]
                
        old_arr = m.vectors_norm
                
        new_arr = np.array([old_arr[index] for index in indices])
        m.vectors_norm = m.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.vocab = new_vocab

    return models

In [ ]:
model_1.init_sims(replace=True)
model_2.init_sims(replace=True)

In [ ]:
models_ali = align_gensim_models([model_1.wv, model_2.wv])

<ipython-input-50-27191591a0df>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  models_ali = align_gensim_models([model_1.wv, model_2.wv])
<ipython-input-48-a84bdbe9e880>:53: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  m.vectors_norm = m.syn0 = new_arr


In [ ]:
model_1_ali = models_ali[0]

In [ ]:
model_2_ali = models_ali[1]

## Measure distances

### For `1` pair of words

In [ ]:
vec_t1 = model_1['person']

In [ ]:
vec_t2 = model_2['person']

In [ ]:
assert vec_t1.shape[0] == 300

In [ ]:
assert vec_t2.shape[0] == 300

In [ ]:
cosine(vec_t1, vec_t2)

### For full vocabulary

In [ ]:
def get_distances(model_1, model_2):
    dist_dict = {}
    # will need to replace `vecs_t1.vocab` with a general vocab
    for word in model_1.wv.vocab:
        if word in model_2.wv.vocab:
            dist_dict[word] = cosine(model_1[word], model_2[word])
        else:
            pass
    dist_dict_df = pd.DataFrame(
        data=dist_dict.items(), 
        columns=['word', 'dist']
    )\
        .sort_values('dist', ascending=False)
    return dist_dict_df

In [ ]:
dists = get_distances(model_1, model_2)

<ipython-input-53-682d38aa6c6f>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  for word in model_1.wv.vocab:
<ipython-input-53-682d38aa6c6f>:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model_2.wv.vocab:


In [ ]:
dists.nlargest(20, 'dist')

,word,dist
1000,virus,0.901883
792,mask,0.898984
13665,subordinate,0.878960
665,mail,0.862455
908,masks,0.859398
15531,characterizing,0.837203
4837,cloth,0.825533
2413,rioters,0.823422
2607,usps,0.789642
15786,gutter,0.786883


### Inspect nearest neighbours

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar('cancel', topn=20):
        print(f'{nb:<15}{model.vocab[nb].count:>6}')
    print('\n---\n')

In [ ]:
for model in [model_1, model_2]:
    i = 0
    while i < 10:
        for nb, dist in model.most_similar(lex):
            if model.vocab[nb].count > 300:
                print(f'{nb:<15}{model.vocab[nb].count:>6}')
                i += 1
    print('\n---\n')

### Temporal variation

In [ ]:
lex = 'trump'

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(nb)
    print('\n---\n')

### Social variation

In [ ]:
lex = 'quarantine'

In [ ]:
print(f'{COMM_1}\n---')
for nb, dist in model_1.most_similar(lex, topn=10):
    print(nb)

In [ ]:
print(f'{COMM_2}\n---')
for nb, dist in model_2.most_similar(lex, topn=10):
    print(nb)

## Obsolete stuff

### Extract vector for target word

### Gensim preprocessing